In [2]:
%load_ext autoreload
%autoreload 2

import helper

## LSTM GAN Training

### PAMAP2

In [4]:
from DeepConvLSTM_model import DeepConvNet
val_model = helper.load_pl_model("val_model.ckpt", DeepConvNet, hidden_size = 256)
helper.train_LSTM_GAN(helper.load_PAMAP2_activity,
                     val_model,
                     start_activity = 1,
                     total_activities = 7,
                     val_iter_size = 3, 
                     batch_size = 20,
                     data_size = (27, 100),
                     noise_len = 100,
                     gen_num_layers = 2,
                     gen_bidirectional = False,
                     dis_hidden_size = 100,
                     dis_num_layers = 2,
                     dis_bidirectional = False,
                     tensorboard_save_dir = "LSTM_GAN_logs",
                     tensorboard_name_prefix = "PAMAP2_act_",
                     )

File exists. Loading
Keep only activity number  1
Windowing
Done!


GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name          | Type          | Params
------------------------------------------------
0 | criterion     | BCELoss       | 0     
1 | generator     | Generator     | 17.6 M
2 | discriminator | Discriminator | 697 K 
3 | val_model     | DeepConvNet   | 3.0 M 
4 | f1            | F1            | 0     
------------------------------------------------
18.3 M    Trainable params
3.0 M     Non-trainable params
21.3 M    Total params
85.081    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name          | Type          | Params
------------------------------------------------
0 | criterion     | BCELoss       | 0     
1 | generator     | Generator     | 17.6 M
2 | discriminator | Discriminator | 697 K 
3 | val_model     | DeepConvNet   | 3.0 M 
4 | f1            | F1            | 0     
------------------------------------------------
18.3 M    Trainable params
3.0 M     Non-trainable params
21.3 M    Total params
85.081    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Success!
{1: 1, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0}


### RWHAR

In [2]:
from DeepConvLSTM_model import DeepConvNet
val_model = helper.load_pl_model("RWHAR_LSTM_918.ckpt", DeepConvNet, 
                                 in_channels = 6,input_size = 50, hidden_size = 768, 
                                 output_size = 8, conv_filter = (5,5), conv_padding = (2,2))
helper.train_LSTM_GAN(helper.load_RWHAR_activity,
                     val_model,
                     start_activity = 1,
                     total_activities = 8,
                     val_iter_size = 3, 
                     batch_size = 64,
                     data_size = (6, 50),
                     noise_len = 100,
                     gen_num_layers = 1,
                     gen_bidirectional = False,
                     dis_hidden_size = 200,
                     dis_num_layers = 2,
                     dis_bidirectional = False,
                     decay = 1,
                     tensorboard_save_dir = "LSTM_GAN_logs",
                     tensorboard_name_prefix = "RWHAR_act_",
                     )


File exists. Loading
Selecting Activity  1
Windowing


GPU available: True, used: True
TPU available: None, using: 0 TPU cores


Done!



  | Name          | Type          | Params
------------------------------------------------
0 | criterion     | BCELoss       | 0     
1 | generator     | Generator     | 2.1 M 
2 | discriminator | Discriminator | 1.0 M 
3 | val_model     | DeepConvNet   | 11.5 M
4 | f1            | F1            | 0     
------------------------------------------------
3.2 M     Trainable params
11.5 M    Non-trainable params
14.6 M    Total params
58.498    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

{1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: None}


## Transformer GAN
### PAMAP2

In [ ]:
from TransformerClassifier import TransformerClassifier
val_model = helper.load_pl_model("val_model_transformer.ckpt", TransformerClassifier, 
                                 in_channels = 27, output_size = 7, 
                                 d_model = 100, nhead = 1, dim_feedforward = 2048,)
helper.train_transformer_GAN(
    helper.load_PAMAP2_activity,
    val_model,
    start_activity = 1,
    total_activities = 7,
    val_iter_size = 3,
    batch_size = 32,
    data_size = (27, 100),
    noise_len = 100,
    period = 100,
    max_retries = 2,
    init_dim_feedforward = 2048,
    dim_feedforward_exponent = 5,
    gen_nheads = 5,
    dis_nheads = 5,
    tensorboard_save_dir = "transformer_GAN_logs",
    tensorboard_name_prefix = "PAMAP2_act_",
    )

### RWHAR

In [3]:
from TransformerClassifier import TransformerClassifier
val_model = helper.load_pl_model("RWHAR_transformer_10layer.ckpt", TransformerClassifier, 
                                 in_channels = 6, output_size = 8, 
                                 d_model = 50, nhead = 5, dim_feedforward = 10000, num_layers = 10)
# from Validation_model import Net
# val_model = Net.load_from_checkpoint("RWHAR_transformer_fixed.ckpt") 

helper.train_transformer_GAN(
    helper.load_RWHAR_activity,
    val_model,
    start_activity = 1,
    total_activities = 8,
    val_iter_size = 3,
    batch_size = 128,
    data_size = (6, 50),
    noise_len = 100,
    period = 100,
    max_retries = 2,
    init_dim_feedforward = 4000,
    dim_feedforward_exponent = 2,
    gen_nheads = 5,
    dis_heads = 5,
    gen_num_layers = 5,
    dis_num_layers = 5,
    tensorboard_save_dir = "transformer_GAN_logs",
    tensorboard_name_prefix = "RWHAR_act_",
    dis_lr = 0.0004,
    gen_lr = 0.0002,
    )

File exists. Loading
Selecting Activity  1
Windowing
Done!
Activity  1 , Try  0


GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name          | Type                  | Params
--------------------------------------------------------
0 | criterion     | BCELoss               | 0     
1 | generator     | Generator             | 4.4 M 
2 | discriminator | Discriminator         | 2.2 M 
3 | val_model     | TransformerClassifier | 10.3 M
4 | f1            | F1                    | 0     
--------------------------------------------------------
6.6 M     Trainable params
10.3 M    Non-trainable params
16.9 M    Total params
67.604    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...


  | Name          | Type                  | Params
--------------------------------------------------------
0 | criterion     | BCELoss               | 0     
1 | generator     | Generator             | 4.4 M 
2 | discriminator | Discriminator         | 2.2 M 
3 | val_model     | TransformerClassifier | 10.3 M
4 | f1            | F1                    | 0     
--------------------------------------------------------
6.6 M     Trainable params
10.3 M    Non-trainable params
16.9 M    Total params
67.604    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Success!
{1: 6, 2: 3, 3: 4, 4: 3, 5: 3, 6: 5, 7: 4, 8: 3}


In [ ]:
## Things to Try to stabilise GAN
## 3. Use one sided label smoothing for discriminator (helps a little)
## 5. Remove Linear and use conv only (helps a ton)
## 6. Use dropout of 0.5 